In [10]:
import pandas as pd
import pandahouse as ph
import seaborn as sns
import numpy as np
import datetime
import io
import telegram
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(16.00,9.00)})

In [2]:
# Настройки бота
bot = telegram.Bot(token='######')
# куда отправляем данные
chat_id = ######

присоединяемся к базам данных и отправляем запросы

In [5]:

connection = {
    'host': '#####',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}

In [11]:
quan_std = 2.5

query = '''select toTime(time) as time, post_users,views,likes,CTR,messages,mess_users
from (select toStartOfFifteenMinutes(time) as time, 
        count(distinct user_id) as post_users, 
        countIf(action = 'view') as views, 
        countIf(action = 'like') as likes, 
        round((countIf(action = 'like')/countIf(action = 'view')*100),1) as CTR
from simulator.feed_actions
where time > today()
group by time
order by time) as t1
full outer join 
(select toTime(time) as time, messages,mess_users
from (select toStartOfFifteenMinutes(time) as time, 
        count(reciever_id) as messages, 
        count(distinct user_id) as mess_users
from simulator.message_actions
where time > today()
group by time
order by time

)) as t2
using time
'''
query_std =f'''select toTime(time) as time,anyLast(post_users) as yd_post_users, avg(post_users)-{quan_std}*stddevSamp(post_users) as low_post_users,avg(post_users)+{quan_std}*stddevSamp(post_users) as high_post_users,
    anyLast(views) as yd_views, avg(views)-{quan_std}*stddevSamp(views) as low_views,avg(views)+{quan_std}*stddevSamp(views) as high_views,
    anyLast(likes) as yd_likes, avg(likes)-{quan_std}*stddevSamp(likes) as low_likes,avg(likes)+{quan_std}*stddevSamp(likes) as high_likes,
    anyLast(CTR) as yd_CTR, avg(CTR)-{quan_std}*stddevSamp(CTR) as low_CTR,avg(CTR)+{quan_std}*stddevSamp(CTR) as high_CTR,
    anyLast(mess_users) as yd_mess_users, avg(mess_users)-{quan_std}*stddevSamp(mess_users) as low_mess_users,avg(mess_users)+{quan_std}*stddevSamp(mess_users) as high_mess_users,
    anyLast(messages) as yd_messages, avg(messages)-{quan_std}*stddevSamp(messages) as low_messages,avg(messages)+{quan_std}*stddevSamp(messages) as high_messages
from (select * from
(select toStartOfFifteenMinutes(time) as time, uniqExact(user_id) as post_users, countIf(action='view' ) as views,countIf(action='like' ) as likes,round(likes/views*100,2)as CTR
from simulator.feed_actions
where time >= today()-7 and toTime(time) <= toTime(now()-900) and time < today()
group by time) as T1
left join
(select uniqExact(user_id) as mess_users, toStartOfFifteenMinutes(time) as time, count(user_id) as messages
from simulator.message_actions
where time >= today()-7 and toTime(time) <= toTime(now()-900) and time < today()
group by time) as T2
using time
order by toTime(time), toDate(time) asc)
group by time
order by time asc'''

In [12]:
df = ph.read_clickhouse(query, connection=connection, index_col='time')
df_std = ph.read_clickhouse(query_std, connection=connection, index_col='time')

Функция проверки данных на откланение от доверительного интервала 

In [15]:
def std(df,graph):
    if df.iloc[-2] > graph.iloc[:, 2].iloc[-1] or df.iloc[-2] < graph.iloc[:, 1].iloc[-1]:


        sns.lineplot(data=df[0:-1], color='orange')
        sns.lineplot(data=graph.iloc[:, 0],linestyle="dashed")
        plt.fill_between(x=graph.index, y1=graph.iloc[:, 1], y2=graph.iloc[:, 2], alpha=0.2)
        plt.title('Alert')
        alert_plot = io.BytesIO()
        plt.tight_layout()
        plt.savefig(alert_plot)
        alert_plot.name = 'Alert_plot.png'
        alert_plot.seek(0)
        plt.close()


        if df.name == 'post_users':
            chart = 'http://superset.lab.karpov.courses/r/193'
            met='Пользователи ленты'
        elif df.name == 'views':
            chart = 'http://superset.lab.karpov.courses/r/194'
            met='Просмотры ленты'
        elif df.name == 'likes':
            chart = 'http://superset.lab.karpov.courses/r/195'
            met='Лайки ленты'
        elif df.name == 'CTR':
            chart = 'http://superset.lab.karpov.courses/r/196'
            met='CTR ленты'
        elif df.name == 'mess_users':
            chart = 'http://superset.lab.karpov.courses/r/197'
            met='Пользователи сообщений'
        elif df.name == 'messages':
            chart = 'http://superset.lab.karpov.courses/r/198'
            met='Количество сообщений'
        else:
            chart = "https://superset.lab.karpov.courses/superset/dashboard/82/"
            met='Метрика'

        bot.sendPhoto(chat_id=chat_id,photo=alert_plot,parse_mode ='HTML', 
                      caption=f'''<b><u>{met}</u></b> в <b><u>{df.index[-1].strftime('%H:%M')}</u></b>.

Текущее значение <b><u>{df.iloc[-1]}</u></b>. Отклонение более <b><u>{round((df.iloc[-1]-df[0:-1].mean())/df[0:-1].mean()*100,1)}%</u></b>.
Глянь <a href="{chart}">на график</a> и <a href="https://superset.lab.karpov.courses/superset/dashboard/82/">в дашборд</a>''')

In [14]:
# проверка каждой метрики
std(df.post_users,df_std[['yd_post_users','low_post_users','high_post_users']])
std(df.likes,df_std[['yd_likes','low_likes','high_likes']])
std(df.views,df_std[['yd_views','low_views','high_views']])
std(df.CTR,df_std[['yd_CTR','low_CTR','high_CTR']])
std(df.mess_users,df_std[['yd_mess_users','low_mess_users','high_mess_users']])
std(df.messages,df_std[['yd_messages','low_messages','high_messages']])